In [47]:
##Code based on lab 3
# -*- coding: utf-8 -*-
import os, re, codecs
import nltk
from nltk.metrics.scores import accuracy
from nltk import sent_tokenize, word_tokenize

from nltk.metrics.distance import edit_distance
from nltk.metrics.distance import jaccard_distance


In [48]:
#Define stop words
stopWords = []

with open('stopWords.txt', "r") as f:
    stopWords = f.read().split()
    
print(stopWords)

['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para', 'é', 'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'foi', 'ao', 'ele', 'das', 'tem', 'à', 'seu', 'sua', 'ou', 'ser', 'quando', 'muito', 'há', 'nos', 'já', 'está', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'era', 'depois', 'sem', 'mesmo', 'aos', 'ter', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'estão', 'você', 'tinha', 'foram', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha', 'têm', 'numa', 'pelos', 'elas', 'havia', 'seja', 'qual', 'será', 'nós', 'tenho', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'fosse', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 'estav

In [49]:
def mainFunction(listaTagsTreino, listaFrasesTreino, listaFrasesDesenvolvimento):
    results = []
    bestSentences = []
    i = 0
    while i < len(listaFrasesDesenvolvimento):
        j = 0
        best = 1000
        tagId = "VOID"
        bestSentence = ""
        while j < len(listaFrasesTreino):
            # It is really a distance and not a similarity measure (1-similarity)
            result = jaccard_distance(set(listaFrasesTreino[j].split()), set(listaFrasesDesenvolvimento[i].split()))
            #result = edit_distance(listaFrasesTreino[j].split(), listaFrasesDesenvolvimento[i].split())
            #print(result)
            if result < best:
                tagId = listaTagsTreino[j]
                bestSentence = listaFrasesTreino[j]
                best = result
        j = j + 1
        results.append(tagId)
        bestSentences.append(bestSentence)
        i = i + 1
    return results, bestSentences

In [50]:
def main():
    
    listaTagsDesenvolvimento = extrai('Corpora/dist-teste.txt',1)
    listaFrasesDesenvolvimento = extrai('Corpora/dist-teste.txt', 2)

    # Process treino
    listaTagsTreino = extrai('Corpora/dist-treino-sem-acentos.txt', 1)
    listaFrasesTreino = extrai('Corpora/dist-treino-sem-acentos.txt', 2)

    #----- Pre-processing-----
    listaFrasesDesenvolvimento = preProc(listaFrasesDesenvolvimento)
    listaFrasesTreino = preProc(listaFrasesTreino)

    #----- Remove stopWords-----
    listaFrasesDesenvolvimento = removeStopWords(listaFrasesDesenvolvimento, stopWords)
    listaFrasesTreino = removeStopWords(listaFrasesTreino, stopWords)

    #----- Stemming -----
    listaFrasesDesenvolvimento = tokStem(listaFrasesDesenvolvimento)
    listaFrasesTreino = tokStem(listaFrasesTreino)

    # Call the main function
    listaTagsEstimada = mainFunction(listaTagsTreino , listaFrasesTreino, listaFrasesDesenvolvimento)[0]
    fraseMaisProxima = mainFunction(listaTagsTreino , listaFrasesTreino, listaFrasesDesenvolvimento)[1]

    # Show results
    for a, b, c, d in zip(listaFrasesDesenvolvimento, listaTagsEstimada, listaTagsDesenvolvimento, fraseMaisProxima):
        print("Sentence to evaluate: ", a)
        print("Suggested Tag: ", b)
        print("Correct Tag: ", c)
        print("Closest sentence: ", d, "\n\n")
    
    # Find accuracy
    print ("Accuracy:", accuracy(listaTagsDesenvolvimento, listaTagsEstimada))


In [51]:
def preProc(Lista):
    perguntas = []
    for l in Lista:
        # ELIMINA ACENTOS
        l = re.sub(u"ã", 'a', l)
        l = re.sub(u"á", "a", l)
        l = re.sub(u"à", "a", l)
        l = re.sub(u"õ", "o", l)
        l = re.sub(u"ô", "o", l)
        l = re.sub(u"ó", "o", l)
        l = re.sub(u"é", "e", l)
        l = re.sub(u"ê", "e", l)
        l = re.sub(u"í", "i", l)
        l = re.sub(u"ú", "u", l)
        l = re.sub(u"ç", "c", l)
        l = re.sub(u"Ã", 'A', l)
        l = re.sub(u"Á", "A", l)
        l = re.sub(u"À", "A", l)
        l = re.sub(u"Õ", "O", l)
        l = re.sub(u"Ô", "O", l)
        l = re.sub(u"Ô", "O", l)
        l = re.sub(u"Ó", 'O', l)
        l = re.sub(u"Í", "I", l)
        l = re.sub(u"Ú", "U", l)
        l = re.sub(u"Ç", "C", l)
        l = re.sub(u"É", "E", l)
        # TUDO EM MINÚSCULAS
        l = l.lower()
        # ELIMINA PONTUAÇÃO
        l = re.sub("[?|\.|!|:|,|;]", '', l)
        # fica so com as perguntas
        l = re.sub("^\w+\t+[^\w]", '', l)
        perguntas.append(str(l))
    return perguntas


In [52]:

# It is case insensitive
def removeStopWords(list, stopWordList):
    perguntas = []
    for sentence in list:
        sentence = sentence.split()
        frase = []
        for word in sentence:
            if word.lower() not in stopWordList:
                frase.append(word)
            fraseAux = ' '.join(frase)	
        perguntas.append(fraseAux)
    return perguntas


In [53]:
def tokStem(perguntas):
    perguntas_tok_stem = []
    stemmer = nltk.stem.RSLPStemmer()
    for l in perguntas:
        l = nltk.word_tokenize(l)
        l1 = []
        for word in l:
            word = stemmer.stem(word)
            l1.append(word)
        l = ' '.join(l1)
        perguntas_tok_stem.append(l)
    return perguntas_tok_stem

In [54]:
#---------------
# Print a list
#---------------
def print_list(list):
    j = 0
    while j < len(list):
        print (list[j])
        j = j + 1

#---------------
# Print both lists, side by side
# They should have the same size
#---------------
def print_lists(list1, list2):
    j = 0
    while j < len(list1):
        print (j, list1[j] + "\t" + list2[j])
        j = j + 1

In [55]:
def extrai(nameFile, numColuna):
    file = open(nameFile, 'rU')
    tags = []
    sentences = []
    for line in file:
        field = re.search(r"(\w+[^\s])\t+(.+)", line)
        if field is None:
            print ("nada")
        else:
            tag = field.group(1)
            sentence = field.group(2)
            tags.append(tag)
            sentences.append(sentence)
    file.close()
    if numColuna == 1:
        return tags
    if numColuna == 2:
        return sentences

In [56]:
main()

nada
nada


/Users/joaotiagoaparicio/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 'U' mode is deprecated
  


KeyboardInterrupt: 